# Installation des dépendances


In [ ]:
!pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install diffusers[torch] transformers accelerate xformers fastai gradio

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

### 📚 Importations
- `torch` : pour le traitement avec PyTorch et le contrôle du générateur aléatoire.
- `StableDiffusionPipeline` & `DPMSolverMultistepScheduler` : composants essentiels de `diffusers` pour générer des images.
- `gradio` : bibliothèque pour créer facilement une interface web interactive.


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import gradio as gr


### ✏️ Fonction `optimize_prompt`
Cette fonction génère :
- un **prompt positif** optimisé selon le produit, la couleur et la description ;
- un **prompt négatif** pour exclure les défauts courants (basse qualité, déformations...).

Elle permet d'améliorer la qualité du rendu généré par le modèle Stable Diffusion.


In [ ]:
def optimize_prompt(product_type, color, description):
    phrasing = {
        "T-shirt": {
            "modif": "illustration détaillée sur t-shirt",
            "style": "style réaliste, haute résolution, éclairage dramatique, rendu artistique, impression de qualité"
        },
        "Mug": {
            "modif": "illustration détaillée sur mug",
            "style": "style réaliste, haute résolution, éclairage doux, rendu artistique, impression de qualité"
        },
        "Casquette": {
            "modif": "illustration brodée sur casquette",
            "style": "style simple, contraste élevé, impression nette, rendu textile"
        },
        "Coussin": {
            "modif": "motif harmonieux sur coussin",
            "style": "style décoratif, haute résolution, textile de qualité"
        },
        "Tableau": {
            "modif": "impression fine art sur toile",
            "style": "style artistique, composition équilibrée, détails fins, galerie d'art, rendu haut de gamme"
        }
    }
    this = phrasing.get(product_type, phrasing["T-shirt"])
    support = f"{this['modif']} {color}"
    optimized_prompt = f"{description}, {support}, {this['style']}"
    negative_prompt = "déformé, basse qualité, pixelisé, flou, proportions incorrectes, anatomie incorrecte, texte illisible, signature, logo, watermark"
    return optimized_prompt, negative_prompt


### ⚙️ Chargement du modèle
- On utilise **Stable Diffusion 2.1** avec précision `float16` pour accélérer l’inférence.
- Le scheduler est remplacé par **DPMSolverMultistep** pour des résultats plus rapides et précis.
- Le pipeline est transféré sur GPU (`cuda`) pour de meilleures performances.


In [ ]:
MODEL_ID = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

### 🖼️ Fonction `generate_image`
Elle permet de :
- Construire un **prompt optimisé** et un **prompt négatif**.
- Générer une image de **768x768 px** avec un nombre d’étapes et un `guidance scale` personnalisable.
- Utiliser un `seed` pour reproductibilité si fourni.


In [ ]:
def generate_image(product_type, color, description, steps, guidance, seed):
    prompt, negative_prompt = optimize_prompt(product_type, color, description)
    try:
        if seed is not None and int(seed) != -1:
            generator = torch.Generator("cuda").manual_seed(int(seed))
        else:
            generator = None
    except Exception:
        generator = None
    result = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=int(steps),
        guidance_scale=float(guidance),
        width=768,
        height=768,
        generator=generator
    )
    image = result.images[0]
    return image, prompt, negative_prompt


### 🧰 Interface Gradio
- Interface responsive avec **Gradio Blocks**.
- Entrées : produit, couleur, description + options avancées (`steps`, `guidance`, `seed`).
- Sorties : image générée, prompt optimisé, prompt négatif.
- `on_generate` : exécute la génération avec les valeurs d'entrée.
- `clear_fields` : réinitialise tous les champs aux valeurs par défaut.
- `generate_btn.click(...)` : connecte le bouton à la fonction de génération.
- `clear_btn.click(...)` : connecte le bouton à la fonction de réinitialisation.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("""
        # 🎨 <span style='color:#8f6ed5'>Générateur d'Images de Produits Personnalisés</span>
        <span style="font-size:1.1em;">
        Saisissez le type de produit, une couleur et votre idée <br>
        Obtenez un rendu <b>photoréaliste</b>&nbsp;: idéal pour vos boutiques en ligne !
        </span>
    """)
    with gr.Row():
        with gr.Column():
            product_type = gr.Dropdown(["T-shirt", "Mug", "Casquette", "Coussin", "Tableau"], label="Type de produit", value="T-shirt")
            color = gr.Dropdown(["blanc", "noir", "bleu", "rouge", "vert"], label="Couleur", value="blanc")
            description = gr.Textbox(label="Description", placeholder="Votre description ici")
            with gr.Accordion("Paramètres avancés", open=False):
                steps = gr.Slider(20, 50, value=30, step=1, label="Étapes de diffusion")
                guidance = gr.Slider(5.0, 9.0, value=7.5, step=0.1, label="Guidance Scale")
                seed = gr.Number(value=-1, label="Seed (-1 pour aléatoire)")
            generate_btn = gr.Button("✨ Générer")
            clear_btn = gr.Button("🧹 Effacer")
        with gr.Column():
            output_image = gr.Image(label="Aperçu produit", type="pil")
            optimized_prompt = gr.Textbox(label="Prompt optimisé réel")
            negative_prompt = gr.Textbox(label="Prompt négatif")

    # Mover les appels .click() à l'intérieur du bloc `with gr.Blocks() as demo:`
    def on_generate(*vals):
        return generate_image(*vals)

    def clear_fields():
        # Restaure les valeurs par défaut
        return ("T-shirt", "blanc", "", 30, 7.5, -1, None, "", "")

    generate_btn.click(
        on_generate,
        inputs=[product_type, color, description, steps, guidance, seed],
        outputs=[output_image, optimized_prompt, negative_prompt]
    )
    clear_btn.click(
        clear_fields,
        None,
        [product_type, color, description, steps, guidance, seed, output_image, optimized_prompt, negative_prompt]
    )



### 🚀 Lancement de l'application Gradio
- L’interface est lancée avec `share=True` pour obtenir un **lien public** accessible depuis n’importe quel navigateur.


In [ ]:
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc830c633d41f94b8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
